# Naifu notebook

## How to run

1. Register ngrok account and get Token.

    https://dashboard.ngrok.com/get-started/your-authtoken

2. Upload this file to Google Drive.
3. Open with Google Colaboratory and set as follows.

    ```
    Runtime > Change runtime type
        Type                  : Python3
        Hardware Accelerator  : GPU
        GPU Type              : A100
    ```
    
4. Assign the acquired token to the NGROK_TOKEN variable.
5. Runtime > Run all cells

### Note

It takes time to process the data download only for the first time.

If you don't subscribe to Google One, you may run out of space on Google Drive.

## Imports

In [ ]:
import os
from google.colab import drive

## User Settings

In [ ]:
NAIFU_PATH = "/content/drive/MyDrive/naifu_main"
NGROK_TOKEN = "<ngrok token here>"

os.environ['ERROR_REPORTING'] = "FALSE"
os.environ['ERROR_REPORTING'] = "FALSE"
os.environ['DTYPE'] = "float32"
os.environ['CLIP_CONTEXTS'] = "3"
os.environ['AMP'] = "1"
os.environ['MODEL'] = "stable-diffusion"
os.environ['DEV'] = "True"
os.environ['ENABLE_EMA'] = "1"
os.environ['PENULTIMATE'] = "1"
os.environ['PYTHONDONTWRITEBYTECODE'] = "1"

os.environ['CONFIG_PATH'] = "models/animefull-final-pruned/config.yaml"
os.environ['MODEL_PATH'] = "models/animefull-final-pruned/model.ckpt"
os.environ['VAE_PATH'] = "models/animevae.pt"

os.environ['SAVE_FILES'] = "1"
os.environ['SAVE_TYPE'] = "full"

## Overhead

In [ ]:
drive.mount('/content/drive')

if not os.path.exists(NAIFU_PATH):
  os.makedirs(NAIFU_PATH)
else:
  print("Directory is already exist.")

%cd {NAIFU_PATH}

## Setup

In [ ]:
def download_torrent(save_path, magnet):
  !pip install --upgrade pip setuptools wheel
  !pip install lbry-libtorrent wget torf
  !apt install python3-libtorrent

  import zipfile
  import os
  import libtorrent as lt
  import time
  import datetime
  import shutil

  params = {
    'save_path': save_path,
    'storage_mode': lt.storage_mode_t(2),
  }

  ses = lt.session()
  ses.listen_on(6881, 6891)
  link = magnet

  if link.endswith('.torrent'):
    import wget
    from torf import Torrent

    if os.path.exists('torrent.torrent'):
      os.remove('torrent.torrent')

    wget.download(link, 'torrent.torrent')
    t = Torrent.read('torrent.torrent')
    link = str(t.magnet(name=True, size=False, trackers=False, tracker=False))


  print(link)
  handle = lt.add_magnet_uri(ses, link, params)
  # change the 0 to a 1 to download sequentially
  handle.set_sequential_download(0)
  ses.start_dht()
  begin = time.time()

  print(datetime.datetime.now())
  print('Downloading Metadata...')

  while not handle.has_metadata():
    time.sleep(1)

  print('Got Metadata, Starting Torrent Download...')
  print("Starting", handle.name())

  while handle.status().state != lt.torrent_status.seeding:
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata',
                'downloading', 'finished', 'seeding', 'allocating']
    print('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' %
          (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
          s.num_peers, state_str[s.state]))
    time.sleep(5)

  end = time.time()
  print(handle.name(), "COMPLETE")
  print("Elapsed Time: ", int((end - begin) // 60), "min :", int((end - begin) % 60), "sec")
  print(datetime.datetime.now())

def setup():
  !git clone https://github.com/Limitex/NAIFU
  download_torrent(NAIFU_PATH, "magnet:?xt=urn:btih:4a4b483d4a5840b6e1fee6b0ca1582c979434e4d&dn=naifu&tr=udp%%3a%%2f%%2ftracker.opentrackr.org%%3a1337%%2fannounce")

  !mkdir {NAIFU_PATH}/NAIFU/models
  !mv {NAIFU_PATH}/naifu/models {NAIFU_PATH}/NAIFU

if len(os.listdir(NAIFU_PATH)) == 0:
  setup()
else:
  print("Setup cannot run because the directory is not empty.")

## Run

In [ ]:
%cd {NAIFU_PATH}/NAIFU

!pip install -r requirements.txt

if os.path.exists("ngrok"):
  !rm ngrok-stable-linux-amd64.zip
  !rm ngrok

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

!./ngrok authtoken {NGROK_TOKEN}
get_ipython().system_raw('./ngrok http 8000 -host-header="127.0.0.1:8000" -region=jp &')

import time
time.sleep(10)

print("\nPublic link: ")
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
print("\nStart server process")
!python -m uvicorn --host 127.0.0.1 --port=8000 --workers 1 main:app